# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
vacation_data = pd.read_csv("weather_data.csv")
vacation_data.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,piracicaba,-22.73,-47.65,71.60,78,40,12.75,BR,1607551566
1,bethel,41.37,-73.41,33.01,95,75,6.93,US,1607551207
2,vaini,-21.20,-175.20,82.40,78,75,13.87,TO,1607551632
3,geraldton,-28.77,114.60,68.00,82,75,1.12,AU,1607551632
4,narsaq,60.92,-46.05,17.96,75,0,8.93,GL,1607551632
5,ushuaia,-54.80,-68.30,48.20,93,90,9.17,AR,1607551292
6,qaanaaq,77.48,-69.36,2.77,87,100,3.49,GL,1607551632
7,punta arenas,-53.15,-70.92,51.80,66,75,6.93,CL,1607551403
8,santa cruz de tenerife,28.47,-16.25,64.00,93,75,8.05,ES,1607551469
9,bredasdorp,-34.53,20.04,60.80,93,50,3.36,ZA,1607551633


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
gmaps.configure(api_key=g_key)

locations = vacation_data[["Lat", "Lng"]].astype(float)
humidity = vacation_data["Humidity"].astype(float)
center = (41.37,-73.41)
fig = gmaps.figure(zoom_level=2, center=center)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
new_vacation = vacation_data.loc[(vacation_data["Max Temp"] < 80) & (vacation_data["Max Temp"] > 70) & (vacation_data["Wind Speed"] < 10) 
                                 & (vacation_data["Cloudiness"] == 0)] 
new_vacation.dropna().head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,viedma,-40.81,-63.00,78.01,28,0,1.99,AR,1607551636
33,rocha,-34.48,-54.33,74.84,63,0,7.16,UY,1607551640
92,clarence town,-32.58,151.78,72.00,72,0,5.82,AU,1607551654
155,nelson bay,-32.72,152.15,72.00,72,0,5.82,AU,1607551710
167,villarrica,-25.75,-56.43,79.32,55,0,5.70,PY,1607551716


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = new_vacation.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
43,chuy,UY,-33.70,-53.46,
85,tokonou,GN,9.65,-9.78,


In [12]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"keyword": "hotel", "type": "hotel", "radius": 5000, "key": g_key}

#iterate through hotel dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f'{lat}, {lng}'
    
    #assemble URL and API request
    print(f'Retrieving results for {index}: {city_name}.')
    response = requests.get(url, params=params).json()
    
    results = response["results"]
    
    try:
        print(f'The closest hotel in {city_name} is {results[0]["name"]}.')
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
          
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")   
    
    time.sleep(1)
          
print("----------------------------")                               

Retrieving results for 43: chuy.
The closest hotel in chuy is Turis Firper Hotel.
Retrieving results for 85: tokonou.
Missing field/result... skipping.
----------------------------


In [13]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,chuy,UY,-33.70,-53.46,Turis Firper Hotel
85,tokonou,GN,9.65,-9.78,


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))